In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from ipywidgets import interactive, Dropdown, Output
from IPython.display import display

In [3]:
import numpy as np
import pandas as pd
import textwrap

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output

import plotly.graph_objects as go

In [4]:
df = pd.read_csv('final_data.csv', low_memory=False)

In [5]:
# df = pd.read_csv('final_data.csv')

In [6]:
# Define conditions for each metric
conditions_urgent = (df['URGNT12MTC_A'] != '0 times') & \
                    (df['URGNT12MTC_A'] != "Don't Know") & \
                    (df['URGNT12MTC_A'] != 'Not Ascertained') & \
                    (df['URGNT12MTC_A'] != 'Refused')

conditions_emerg = (df['EMERG12MTC_A'] != '0 times') & \
                   (df['EMERG12MTC_A'] != "Don't Know") & \
                   (df['EMERG12MTC_A'] != 'Not Ascertained') & \
                   (df['EMERG12MTC_A'] != 'Refused')

conditions_usualpl = (df['USUALPL_A'] == 'Yes') | \
                     (df['USUALPL_A'] == 'There is MORE THAN ONE place')

# Now combine all conditions
all_conditions = conditions_urgent | conditions_emerg | conditions_usualpl

# Apply the combined conditions to the DataFrame
filtered_data = df[all_conditions]

# Output the filtered data
print(filtered_data)

        RECTYPE  SRVY_YR      HHX     WTFA_A               URBRRL REGION  \
0            10     2022  H059086   4548.583   Large Fringe Metro  South   
1            10     2022  H054049   7087.431      Nonmetropolitan  South   
2            10     2022  H055201   8125.516      Nonmetropolitan  South   
3            10     2022  H044893   7837.390      Nonmetropolitan  South   
4            10     2022  H043149  10234.356  Large Central Metro  South   
...         ...      ...      ...        ...                  ...    ...   
120693       10     2021  H012375   3203.634      Nonmetropolitan   West   
120694       10     2021  H052160   1556.049      Nonmetropolitan   West   
120695       10     2021  H051563   6238.789      Nonmetropolitan   West   
120696       10     2021  H058432   4769.800      Nonmetropolitan   West   
120697       10     2021  H052354   6336.260      Nonmetropolitan   West   

        HHRESPSA_FLG  INTV_QRT  INTV_MON                HHSTAT_A  ...  \
0             

In [7]:
# Calculate the percentages for each metric
percentage_urgent = (conditions_urgent.sum() / len(df)) * 100
percentage_emerg = (conditions_emerg.sum() / len(df)) * 100
percentage_usualpl = (conditions_usualpl.sum() / len(df)) * 100

# Print the results
print(f"Percentage with at least one urgent care visit (URGNT12MTC_A): {percentage_urgent:.2f}%")
print(f"Percentage with at least one emergency room visit (EMERG12MTC_A): {percentage_emerg:.2f}%")
print(f"Percentage with a usual place of care (USUALPL_A): {percentage_usualpl:.2f}%")

Percentage with at least one urgent care visit (URGNT12MTC_A): 27.69%
Percentage with at least one emergency room visit (EMERG12MTC_A): 19.72%
Percentage with a usual place of care (USUALPL_A): 90.73%


In [8]:
def calculate_percentage_by_year(metric, condition):
    # Group the data by the metric and year, then apply the condition
    grouped = df.groupby([metric, 'SRVY_YR'])
    
    # Calculate the total entries per year in the overall DataFrame
    total_per_year = df.groupby('SRVY_YR').size()

    # Calculate percentages
    percentages = grouped.apply(lambda x: (condition[x.index].sum() / total_per_year[x.name[1]]) * 100)

    # Convert the result into a DataFrame with years as columns
    return percentages.unstack(level=-1)

In [9]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Interactive Data Visualization'),
    html.P("Select question:"),
    dcc.Dropdown(
        id="question-dropdown",
        options=[
            {'label': 'Urgent care visit', 'value': 'urgent care visit'},
            {'label': 'Emergency room visit', 'value': 'emergency room visit'},
            {'label': 'Usual place of care', 'value': 'usual place of care'}
        ],
        value='urgent care visit',
        clearable=False,
    ),
    html.P("Select metric:"),
    dcc.Dropdown(
        id="metric-dropdown",
        options=[
            {'label': 'URBRRL', 'value': 'URBRRL'},
            {'label': 'REGION', 'value': 'REGION'},
            {'label': 'SEX_A', 'value': 'SEX_A'},
            {'label': 'AGE_FINAL_A', 'value': 'AGE_FINAL_A'},
            {'label': 'RACEALLP_A', 'value': 'RACEALLP_A'},
            {'label': 'MARSTAT_A', 'value': 'MARSTAT_A'},
            {'label': 'NATUSBORN_A', 'value': 'NATUSBORN_A'},
            {'label': 'AFVET_A', 'value': 'AFVET_A'},
            {'label': 'EMPWRKLSW1_A', 'value': 'EMPWRKLSW1_A'},
            {'label': 'EMPWKHRS3_FINAL_A', 'value': 'EMPWKHRS3_FINAL_A'},
            {'label': 'EDUCP_A', 'value': 'EDUCP_A'},
            {'label': 'PCNTADTWKP_A', 'value': 'PCNTADTWKP_A'},
            {'label': 'INCWRKO_A', 'value': 'INCWRKO_A'},
            {'label': 'INCINTER_A', 'value': 'INCINTER_A'},
            {'label': 'RATCAT_A', 'value': 'RATCAT_A'},
            {'label': 'NOTCOV_A', 'value': 'NOTCOV_A'},
            {'label': 'COVER_A', 'value': 'COVER_A'},
            {'label': 'COVER65_A', 'value': 'COVER65_A'},
            {'label': 'MEDICARE_A', 'value': 'MEDICARE_A'},
            {'label': 'MCPART_A', 'value': 'MCPART_A'},
            {'label': 'MEDICAID_A', 'value': 'MEDICAID_A'},
            {'label': 'PRIVATE_A', 'value': 'PRIVATE_A'},
            {'label': 'PLNWRKR1_A', 'value': 'PLNWRKR1_A'},
            {'label': 'CHIP_A', 'value': 'CHIP_A'},
            {'label': 'OTHPUB_A', 'value': 'OTHPUB_A'},
            {'label': 'OTHGOV_A', 'value': 'OTHGOV_A'},
            {'label': 'MILITARY_A', 'value': 'MILITARY_A'},
            {'label': 'IHS_A', 'value': 'IHS_A'},
            {'label': 'RSNHICOST_A', 'value': 'RSNHICOST_A'},
            {'label': 'RSNHIWANT_A', 'value': 'RSNHIWANT_A'},
            {'label': 'RSNHICONF_A', 'value': 'RSNHICONF_A'},
            {'label': 'PAYBLL12M_A', 'value': 'PAYBLL12M_A'},
            {'label': 'PAYWORRY_A', 'value': 'PAYWORRY_A'},
            {'label': 'MEDNG12M_A', 'value': 'MEDNG12M_A'},
            {'label': 'RXDG12M_A', 'value': 'RXDG12M_A'},
            {'label': 'MHTHND_A', 'value': 'MHTHND_A'},
            {'label': 'LASTDR_A', 'value': 'LASTDR_A'},
            {'label': 'WELLNESS_A', 'value': 'WELLNESS_A'},
            {'label': 'USPLKIND_A', 'value': 'USPLKIND_A'},
            {'label': 'DENPREV_A', 'value': 'DENPREV_A'},
            {'label': 'SHTFLU12M_A', 'value': 'SHTFLU12M_A'},
            {'label': 'ANGEV_A', 'value': 'ANGEV_A'},
            {'label': 'MIEV_A', 'value': 'MIEV_A'},
            {'label': 'CHDEV_A', 'value': 'CHDEV_A'},
            {'label': 'CHLEV_A', 'value': 'CHLEV_A'},
            {'label': 'HYPEV_A', 'value': 'HYPEV_A'},
            {'label': 'COPDEV_A', 'value': 'COPDEV_A'},
            {'label': 'ASEV_A', 'value': 'ASEV_A'},
            {'label': 'CANEV_A', 'value': 'CANEV_A'},
            {'label': 'ARTHEV_A', 'value': 'ARTHEV_A'},
            {'label': 'DEMENEV_A', 'value': 'DEMENEV_A'},
            {'label': 'DIBEV_A', 'value': 'DIBEV_A'},
            {'label': 'BMICAT_A', 'value': 'BMICAT_A'},
            {'label': 'ANXFREQ_A', 'value': 'ANXFREQ_A'},
            {'label': 'ANXLEVEL_A', 'value': 'ANXLEVEL_A'},
            {'label': 'DEPFREQ_A', 'value': 'DEPFREQ_A'},
            {'label': 'DEPLEVEL_A', 'value': 'DEPLEVEL_A'},
            {'label': 'MHRX_A', 'value': 'MHRX_A'},
            {'label': 'MHTHRPY_A', 'value': 'MHTHRPY_A'},
            {'label': 'MHTHDLY_A', 'value': 'MHTHDLY_A'},
            {'label': 'PHQ81_A', 'value': 'PHQ81_A'},
            {'label': 'PHQ82_A', 'value': 'PHQ82_A'},
            {'label': 'PHQ86_A', 'value': 'PHQ86_A'},
            {'label': 'PHQ87_A', 'value': 'PHQ87_A'},
            {'label': 'PHQ88_A', 'value': 'PHQ88_A'},
            {'label': 'PHSTAT_A', 'value': 'PHSTAT_A'},
            {'label': 'SOCERRNDS_A', 'value': 'SOCERRNDS_A'},
            {'label': 'SOCSCLPAR_A', 'value': 'SOCSCLPAR_A'},
            {'label': 'SOCWRKLIM_A', 'value': 'SOCWRKLIM_A'},
            {'label': 'HEARINGDF_A', 'value': 'HEARINGDF_A'},
            {'label': 'DIFF_A', 'value': 'DIFF_A'},
            {'label': 'VISIONDF_A', 'value': 'VISIONDF_A'},
            {'label': 'COMDIFF_A', 'value': 'COMDIFF_A'},
            {'label': 'COGMEMDFF_A', 'value': 'COGMEMDFF_A'},
            {'label': 'UPPSLFCR_A', 'value': 'UPPSLFCR_A'}
        ],
        value='URBRRL',
        clearable=False,
    ),
    dcc.Graph(id="interactive-graph"),
])

@app.callback(
    Output("interactive-graph", "figure"), 
    [Input("question-dropdown", "value"),
     Input("metric-dropdown", "value")]
)
def update_graph(question, metric):
    # Map question to condition
    condition_map = {
        'urgent care visit': conditions_urgent,
        'emergency room visit': conditions_emerg,
        'usual place of care': conditions_usualpl
    }
    condition = condition_map[question]
    
    # Calculate percentages by year
    percentages_by_year = calculate_percentage_by_year(metric, condition)
    
    # If the metric is SEX_A, filter the data to only include Male and Female
    if metric == 'SEX_A':
        percentages_by_year = percentages_by_year.loc[['Male', 'Female']]
    # Filter for AGE_FINAL_A
    if metric == 'AGE_FINAL_A':
        valid_ages = ['18-44', '45-64', '65-74', '75 and over']
        percentages_by_year = percentages_by_year.loc[valid_ages]
    if metric == 'RACEALLP_A':
        valid_races = ['AIAN and any other group', 'AIAN only', 'Asian only', 'Black/African American only', 'Other single and multiple races','White only']
        percentages_by_year = percentages_by_year.loc[valid_races]
    if metric == 'NATUSBORN_A':
        valid_usborn = ['Yes', 'No']
        percentages_by_year = percentages_by_year.loc[valid_usborn]
    if metric == 'AFVET_A':
        valid_servedmilitary = ['Yes', 'No']
        percentages_by_year = percentages_by_year.loc[valid_servedmilitary]
    if metric == 'EMPWRKLSW1_A':
        valid_emplastweek = ['Yes', 'No']
        percentages_by_year = percentages_by_year.loc[valid_emplastweek]
    if metric == 'EMPWKHRS3_FINAL_A':
        valid_hoursworked = ['1-20 hours', '21-40 hours','41-60 hours','60-95 hours','95+ hours']
        percentages_by_year = percentages_by_year.loc[valid_hoursworked]
    if metric == 'EDUCP_A':
        valid_educlevel = ['12th grade, no diploma', 'Associate degree: academic program','Associate degree: occupational, technical, or vocational program','Bachelors degree','GED or equivalent','Grade 1-11','High School Graduate' ]
        percentages_by_year = percentages_by_year.loc[valid_educlevel]
    if metric == 'PCNTADTWKP_A':
        valid_adultsinfamworking = ['0 adults', '1 adult','2 adults','3+ adults']
        percentages_by_year = percentages_by_year.loc[valid_adultsinfamworking]
    if metric == 'INCWRKO_A':
        valid_incfromwages = ['Yes', 'No']
        percentages_by_year = percentages_by_year.loc[valid_incfromwages]       
    if metric == 'INCINTER_A':
        valid_incfromaccounts = ['Yes', 'No']
        percentages_by_year = percentages_by_year.loc[valid_incfromaccounts]         
    if metric == 'NOTCOV_A':
        valid_coveredbyins = ['Covered', 'Not Covered']
        percentages_by_year = percentages_by_year.loc[valid_coveredbyins]        
    if metric == 'COVER_A':
        valid_typecovunder65 = ['Medicaid and other public', 'Other Coverage','Private','Uninsured']
        percentages_by_year = percentages_by_year.loc[valid_typecovunder65]         
    if metric == 'COVER65_A':
        valid_typecovover65 = ['Medicare Advantage','Medicare only excluding Medicare Advantage','Dual Eligible', 'Other Coverage','Private','Uninsured']
        percentages_by_year = percentages_by_year.loc[valid_typecovover65]        
    if metric == 'MEDICARE_A':
        valid_havemedicare = ['Yes, information','Yes, but no information', 'No']
        percentages_by_year = percentages_by_year.loc[valid_havemedicare]        
    if metric == 'MCPART_A':
        valid_medicareparts = ['Both Part A and B','Part A - hospital only', 'Part B - medical only']
        percentages_by_year = percentages_by_year.loc[valid_medicareparts]        
    if metric == 'MEDICAID_A':
        valid_havemedicaid = ['Yes, information','Yes, but no information', 'No']
        percentages_by_year = percentages_by_year.loc[valid_havemedicaid]         
    if metric == 'PRIVATE_A':
        valid_haveprivins = ['Yes, information','Yes, but no information', 'No']
        percentages_by_year = percentages_by_year.loc[valid_haveprivins]        
    if metric == 'PLNWRKR1_A':
        valid_howplanobtain = ['Purchased directly','Through an employer, union, or professional organization', 'Through a state or local government or community program','Other','Through Healthcare.gov or the Affordable Care Act, also known as Obamacare','Through parents','Through school']
        percentages_by_year = percentages_by_year.loc[valid_howplanobtain] 
    if metric == 'CHIP_A':
        valid_chip = ['Yes, information','No']
        percentages_by_year = percentages_by_year.loc[valid_chip]         
    if metric == 'OTHPUB_A':
        valid_othpub = ['Yes, information','No']
        percentages_by_year = percentages_by_year.loc[valid_othpub]         
    if metric == 'OTHGOV_A':
        valid_othgov = ['Yes, information','No']
        percentages_by_year = percentages_by_year.loc[valid_othgov]         
    if metric == 'MILITARY_A':
        valid_militaryins = ['Yes, information','Yes, but no information', 'No']
        percentages_by_year = percentages_by_year.loc[valid_militaryins]        
    if metric == 'IHS_A':
        valid_ihs = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_ihs]        
    if metric == 'RSNHICOST_A':
        valid_noinscost = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_noinscost]         
    if metric == 'RSNHIWANT_A':
        valid_noinswant = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_noinswant]        
    if metric == 'RSNHICONF_A':
        valid_noinsconfuse = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_noinsconfuse]        
    if metric == 'PAYBLL12M_A':
        valid_probpaybill = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_probpaybill] 
    if metric == 'PAYWORRY_A':
        valid_worrypaybill = ['Not at all worried','Somewhat worried','Very worried']
        percentages_by_year = percentages_by_year.loc[valid_worrypaybill]        
    if metric == 'MEDNG12M_A':
        valid_dngmedcarecost = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_dngmedcarecost]        
    if metric == 'RXDG12M_A':
        valid_dngpresmedcost = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_dngpresmedcost]         
    if metric == 'MHTHND_A':
        valid_dngtherapycost = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_dngtherapycost]         
    if metric == 'LASTDR_A':
        valid_lastsawdoc = ['Within the past year','Within the last 2 years','Within the last 3 years','Within the last 5 years','Within the last 10 years','10 years ago or more','Never']
        percentages_by_year = percentages_by_year.loc[valid_lastsawdoc]         
    if metric == 'WELLNESS_A':
        valid_wellvisit = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_wellvisit]        
    if metric == 'USPLKIND_A':
        valid_usualplacetype = ["A doctor's office or health center",'Urgent care center or clinic in a drug store or grocery store','A VA Medical Center or VA outpatient clinic','Some other place','Hospital emergency room','Does not go to one place most often']
        percentages_by_year = percentages_by_year.loc[valid_usualplacetype]        
    if metric == 'DENPREV_A':
        valid_lastdentalvisit = ['Within the past year','Within the last 2 years','Within the last 3 years','Within the last 5 years','Within the last 10 years','10 years ago or more','Never']
        percentages_by_year = percentages_by_year.loc[valid_lastdentalvisit]        
    if metric == 'SHTFLU12M_A':
        valid_flushot = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_flushot]
    if metric == 'ANGEV_A':
        valid_angev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_angev]        
    if metric == 'MIEV_A':
        valid_miev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_miev]        
    if metric == 'CHDEV_A':
        valid_chdev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_chdev]        
    if metric == 'CHLEV_A':
        valid_chlev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_chlev]        
    if metric == 'HYPEV_A':
        valid_hypev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_hypev]        
    if metric == 'COPDEV_A':
        valid_copdev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_copdev]        
    if metric == 'ASEV_A':
        valid_asev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_asev]        
    if metric == 'CANEV_A':
        valid_canev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_canev]        
    if metric == 'ARTHEV_A':
        valid_arthev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_arthev]        
    if metric == 'DEMENEV_A':
        valid_demev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_demev]        
    if metric == 'DIBEV_A':
        valid_dibev = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_dibev]
    if metric == 'BMICAT_A':
        valid_bmicat = ['Healthy weight','Obese','Overweight','Underweight']
        percentages_by_year = percentages_by_year.loc[valid_bmicat]          
    if metric == 'ANXFREQ_A':
        valid_anxfreq = ['Daily','Weekly','Monthly','A few times a year','Never']
        percentages_by_year = percentages_by_year.loc[valid_anxfreq]         
    if metric == 'ANXLEVEL_A':
        valid_anxlevel = ['A little','Somewhere in between a little and a lot','A lot']
        percentages_by_year = percentages_by_year.loc[valid_anxlevel]        
    if metric == 'DEPFREQ_A':
        valid_depfreq = ['Daily','Weekly','Monthly','A few times a year','Never']
        percentages_by_year = percentages_by_year.loc[valid_depfreq]         
    if metric == 'DEPLEVEL_A':
        valid_deplevel = ['A little','Somewhere in between a little and a lot','A lot']
        percentages_by_year = percentages_by_year.loc[valid_deplevel]        
    if metric == 'MHRX_A':
        valid_medsformenhealth = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_medsformenhealth]        
    if metric == 'MHTHRPY_A':
        valid_menhealththerapy = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_menhealththerapy]        
    if metric == 'MHTHDLY_A':
        valid_dlythrpycost = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_dlythrpycost]        
    if metric == 'PHQ81_A':
        valid_nointdoingthings = ['Not at all','Several days','More than half the days','Nearly every day']
        percentages_by_year = percentages_by_year.loc[valid_nointdoingthings]         
    if metric == 'PHQ82_A':
        valid_feelingdown = ['Not at all','Several days','More than half the days','Nearly every day']
        percentages_by_year = percentages_by_year.loc[valid_feelingdown]         
    if metric == 'PHQ86_A':
        valid_failureself = ['Not at all','Several days','More than half the days','Nearly every day']
        percentages_by_year = percentages_by_year.loc[valid_failureself] 
    if metric == 'PHQ87_A':
        valid_troublefocus = ['Not at all','Several days','More than half the days','Nearly every day']
        percentages_by_year = percentages_by_year.loc[valid_troublefocus]         
    if metric == 'PHQ88_A':
        valid_movespeakslow = ['Not at all','Several days','More than half the days','Nearly every day']
        percentages_by_year = percentages_by_year.loc[valid_movespeakslow]         
    if metric == 'PHSTAT_A':
        valid_genhealth = ['Poor','Fair','Good','Very Good','Excellent']
        percentages_by_year = percentages_by_year.loc[valid_genhealth]         
    if metric == 'SOCERRNDS_A':
        valid_socialerrands = ['No difficulty','Some difficulty','A lot of difficulty','Cannot do at all']
        percentages_by_year = percentages_by_year.loc[valid_socialerrands]         
    if metric == 'SOCSCLPAR_A':
        valid_socpar = ['No difficulty','Some difficulty','A lot of difficulty','Cannot do at all']
        percentages_by_year = percentages_by_year.loc[valid_socpar]          
    if metric == 'SOCWRKLIM_A':
        valid_wrklim = ['Yes','No']
        percentages_by_year = percentages_by_year.loc[valid_wrklim]         
    if metric == 'HEARINGDF_A':
        valid_diffhearing = ['No difficulty','Some difficulty','A lot of difficulty','Cannot do at all']
        percentages_by_year = percentages_by_year.loc[valid_diffhearing]        
    if metric == 'DIFF_A':
        valid_diffwalksteps = ['No difficulty','Some difficulty','A lot of difficulty','Cannot do at all']
        percentages_by_year = percentages_by_year.loc[valid_diffwalksteps]        
    if metric == 'VISIONDF_A':
        valid_diffseeing = ['No difficulty','Some difficulty','A lot of difficulty','Cannot do at all']
        percentages_by_year = percentages_by_year.loc[valid_diffseeing]        
    if metric == 'COMDIFF_A':
        valid_diffcommunicating = ['No difficulty','Some difficulty','A lot of difficulty','Cannot do at all']
        percentages_by_year = percentages_by_year.loc[valid_diffcommunicating]        
    if metric == 'COGMEMDFF_A':
        valid_diffremembering = ['No difficulty','Some difficulty','A lot of difficulty','Cannot do at all']
        percentages_by_year = percentages_by_year.loc[valid_diffremembering]        
    if metric == 'UPPSLFCR_A':
        valid_diffselfcare = ['No difficulty','Some difficulty','A lot of difficulty','Cannot do at all']
        percentages_by_year = percentages_by_year.loc[valid_diffselfcare]                
        
        
    # Create the figure
    fig = go.Figure()
    for category, values in percentages_by_year.iterrows():
        fig.add_trace(go.Scatter(
            x=percentages_by_year.columns,
            y=values,
            mode='lines+markers',
            name=category
        ))
    
    # Formatting the title and the rest of the plot
    title = f"Percent of people with at least one {question} by {metric} over time"
    fig.update_layout(
        title=title,
        xaxis_title="Year",
        yaxis_title="Percentage (%)"
    )
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


In [10]:
def calculate_percentage_by_year(metric, condition):
    grouped = df.groupby([metric, 'SRVY_YR'])
    total_per_year = df.groupby('SRVY_YR').size()
    percentages = grouped.apply(lambda x: (condition[x.index].sum() / total_per_year[x.name[1]]) * 100)
    return percentages.unstack(level=-1)

def plot_metric_yearly(metric, question):
    if question == 'urgent care visit':
        condition = conditions_urgent
    elif question == 'emergency room visit':
        condition = conditions_emerg
    elif question == 'usual place of care':
        condition = conditions_usualpl
    
    percentages_by_year = calculate_percentage_by_year(metric, condition)
    return percentages_by_year


In [11]:
metric_to_test = 'REGION'  # Replace 'REGION' with the actual metric column name if different
question_to_test = 'emergency room visit'  # Choose based on the conditions defined

# Now let's test the plotting function (or you can directly call calculate_percentage_by_year to just see the data)
percentages_by_year = plot_metric_yearly(metric_to_test, question_to_test)
print(percentages_by_year)

SRVY_YR        2019      2020      2021      2022
REGION                                           
Midwest    4.906710  4.140269  4.073672  4.350656
Northeast  3.831609  3.180436  2.764399  3.160826
South      8.482045  7.045109  6.973747  7.862283
West       4.759821  4.156107  4.389119  4.715923


In [12]:
metric_to_test = 'SEX_A'  # Replace 'REGION' with the actual metric column name if different
question_to_test = 'urgent care visit'  # Choose based on the conditions defined

# Now let's test the plotting function (or you can directly call calculate_percentage_by_year to just see the data)
percentages_by_year = plot_metric_yearly(metric_to_test, question_to_test)
print(percentages_by_year)

SRVY_YR          2019       2020       2021       2022
SEX_A                                                 
Don't Know        NaN   0.000000   0.000000   0.007233
Female      16.420289  15.335150  15.043077  18.057213
Male        11.535456  10.897111  10.765891  12.994105
Refused      0.003125   0.000000   0.000000   0.003617
